In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
(x_train, y_train), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_data = x_test[:2000].astype('float32') / 255.0
x_flat = x_data.reshape((2000, -1))

In [ ]:

pca_results = PCA(n_components=2).fit_transform(x_flat)
ml_labels = KMeans(n_clusters=10, n_init=10).fit_predict(pca_results)